In [1]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI

import gradio as gr

In [2]:
# Define this variable and then pass js=force_dark_mode when creating the Interface


force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [3]:
# Load environment variables in a file callled .env

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
# Connect to OpenAI, Anthropic and Google

openai = OpenAI()


system_message = "You are customer support/assistant for Skechers (a sport shoe company) official website. \
Give the price as a range and add between $5 to $30 to whatever price of shoe you give them  but don't mention it during your answer. \
This extra addition in price will argument for prices lower than actual price."

In [5]:
# Function definition
def stream_assistant(message, history):
    messages = [{"role": "system", "content": system_message}]
    messages.extend(history)  # history is already in [{"role": ..., "content": ...}, ...] format
    messages.append({"role": "user", "content": message})

    stream = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=True
    )
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

In [6]:
gr.ChatInterface(fn=stream_assistant, type="messages", js=force_dark_mode).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
